In [2]:
import numpy as np
import pandas as pd

In [3]:
data_dicts = r"/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60"

In [4]:
import os
dicts_list = [name for name in os.listdir(data_dicts) if not name.endswith('.zip') and name.startswith('data')]

In [5]:
dicts_list

['data4', 'data2', 'data3', 'data1']

Get image folders and label txt files

In [6]:
output_paths = {}
for _dict in dicts_list:
    _folder = data_dicts + '/' + _dict
    img_folders = [name for name in os.listdir(_folder) if not name.endswith('.txt') and not name.startswith('README')]
    for _img_folder in img_folders:
        img_folder = _folder + '/' + _img_folder
        _label_txt = img_folder + '.txt'
        try:
            t = open(_label_txt, 'r')
            t.close()
        except:
            continue
        output_paths[_img_folder] = [img_folder, _label_txt]
print(output_paths)

{'0512151934': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512151934', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512151934.txt'], '0512160254': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512160254', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512160254.txt'], '0512152416': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512152416', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512152416.txt'], '0512150451': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512150451', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512150451.txt'], '0512155226': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512155226', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512155226.txt'], '0512152013': ['/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/0512152013', '/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-60/data4/051215201

Get person ID and the corresponding action label str

In [10]:
dict_label = {}
for t, _dict in enumerate(dicts_list):
    _folder = data_dicts + '/' + _dict
    act_txt = [name for name in os.listdir(_folder) if name.startswith('activity')]
    act_txt = _folder + '/' + act_txt[0]
    label = open(act_txt, 'r')
    my_txt = [txtline[:].split(',') for txtline in label.readlines()]
    label.close()
    for label_data in my_txt[:-1]:
        folder_ID = label_data.pop(0)
        act_label = label_data.pop(0)
        dict_label[folder_ID] = {'person_ID':t+1, 'action_label':act_label}

Generate the dictionary which contains all data information

In [11]:
dict_folders = {}#folder_ID:dict_folder
dict_folder = {}#{person_ID:, dict_frames:, floder_path:, action_label:}
dict_frames = {}#frame_1:{dict_ROI, dict_P}, frame_2:{dict_ROI, dict_P},...,frame_ID:{dict_ROI, dict_P}
dict_ROI = {}#{1:{orin, CONF},2:{},...,11:{}}
dict_P = {}#{1:{coor, CONF},2:{},3:{},...15:{}}

In [12]:
for i, paths in enumerate(output_paths):
    dict_folder = {}
    _list = output_paths[paths]
    folder_path = _list[0]
    folder_ID = paths
    person_ID = dict_label[paths]['person_ID']
    action_label = dict_label[paths]['action_label']
    dict_folder['person_ID'] = person_ID
    dict_folder['folder_path'] = folder_path
    dict_folder['action_label'] = action_label
    
    _txt = open(_list[1], 'r')
    my_txt = [txtline[:-1].split(',') for txtline in _txt.readlines()]
    _txt.close()
    dict_frames = {}
    for _frame_data in my_txt[:-1]:
        frame_ID = int(_frame_data.pop(0))
        dict_ROI, dict_P = {}, {}
        for times in range(15):
            if times < 11:
                ORI_i = []
                for t in range(9):
                    ORI_i.append(float(_frame_data.pop(0)))
                ORI_i_CONF = int(_frame_data.pop(0))
                dict_ROI[times] = {'orin':ORI_i, 'CONF':ORI_i_CONF}
            P_i = []
            for t in range(3):
                P_i.append(float(_frame_data.pop(0)))
            P_i_CONF = int(_frame_data.pop(0))
            dict_P[times] = {'coor':P_i, 'CONF':P_i_CONF}         
        dict_frames[frame_ID] = {'ROI':dict_ROI, 'P':dict_P}
    dict_folder['frames'] = dict_frames
    dict_folders[paths] = dict_folder

Writing to a json file to store the dictionary style data

In [13]:
import json
with open('./data.json', 'w') as json_file:
    json.dump(dict_folders, json_file, indent=4)
json_file.close()

Evaluate the data

In [11]:
len(dict_folders['0510165136']['frames'])

1637

In [12]:
len(dict_folders['0510165136'])

4

****************************